In [1]:
import pandas as pd
import dtale


In [2]:
df = pd.read_csv(r'creating_the_clinic_dataset/preprocessed_data/merged_2021_and_2022.csv')

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (0,44,50,73,84,309,437,442,443,450,457,486,502,540,549,568,728,733,745,747,752,763,801,810,829,882,883,884,885,886,887,888,889,890,891,892,951,989,994,1000,1015,1045,1065,1066,1070,1071,1072,1073,1075,1076,1077,1080,1081,1098,1103,1119,1121,1172,1174,1193,1195,1196,1197,1203,1204,1205,1206,1358,1360,1361,1362,1368,1369,1370,1371,1486,1492,1493,1496,1497,1498,1499,1508,1516,1532,1535,1546) have mixed types.Specify dtype option on import or set low_memory=False.



In [3]:
df['Group'] = 0
df.loc[df.group___1 == 1, 'Group'] = 1
df.loc[df.group___2 == 1, 'Group'] = 2
df.loc[df.group___3 == 1, 'Group'] = 3
#df = df_[df_.Group > 0]
#df.groupby('redcap_event_name').id.count()

In [4]:
def impute_group(df, id_num):
    df.query(f"id ==  '{id_num}'")[['Group', 'redcap_event_name']]
    try:
        correct_group = df.query(f"(id ==  '{id_num}') & (redcap_event_name =='intake_arm_1')").Group.values[0]
        df.loc[df.id ==  id_num, 'Group'] = correct_group
    except IndexError:
        pass
    return df

for id_num in df.id.unique():
    df = impute_group(df, id_num)

In [5]:
cols = list(df.columns)
groups = [i for i in list(cols) if 'group' in i]

info_columns = ['id', 'gender', 'redcap_event_name', 'age_child_pre', 'Group'] + groups


In [6]:

suicidal_behavior_intake = [
    'c_ssrs_t_11_life_clin',
    'c_ssrs_t_12_life_clin',
    'c_ssrs_t_13_life_clin',
    'c_ssrs_t_14_life_clin',
    'c_ssrs_t_15_life_clin',

    'c_ssrs_6_life', 'mfq_34', 'mfq_36', 'c_ssrs_6_2weeks',
    'c_ssrs_6_3month'
]


suicidal_behavior_time2 = [
    'suicidal_behavior_last_11_clin',
    'suicidal_behavior_last_12_clin',
    'suicidal_behavior_last_13_clin',
    'suicidal_behavior_last_14_clin',
    'suicidal_behavior_last_15_clin',

    'c_ssrs_6', 'mfq_36',
    'c_ssrs_last_visit_6',
    'chameleon_behavior_stu',
    'chameleon_attempt_stu',
]

df = df[suicidal_behavior_time2 + suicidal_behavior_intake + info_columns]

pathology_variables_times = {
    'intake': {'suicidal_behavior': suicidal_behavior_intake},
    'time2': {'suicidal_behavior': suicidal_behavior_time2}
}

class TargetVariable:
    def __init__(self, name, info, columns, do_preprocessing=True):

        self.info = info
        self.name = name
        self.columns = columns
        self.do_preprocessing = do_preprocessing

    def calculate_value(self, df):
        if self.do_preprocessing:
            aligned_df = df[self.columns].replace(2, 0)
            df[self.name] = (aligned_df.sum(axis=1) > 0).astype(int)
        else:
            df[self.name] = (df[self.columns].sum(axis=1) > 0).astype(int)
        return df


In [7]:
events = ['intake_arm_1', 'control_5weeks_arm_1']

df_time1 = df[df.redcap_event_name == events[0]]
print(f"{df_time1.shape =}")


df_time2 = df[df.redcap_event_name == events[1]]
print(f"{df_time2.shape = }")

df_time1.shape =(408, 28)
df_time2.shape = (218, 28)


In [8]:

for questionnaire_name, items in pathology_variables_times['intake'].items():
    tv = TargetVariable(questionnaire_name, {'measurement': 'time1'}, items)
    df_time1 = tv.calculate_value(df_time1)

for questionnaire_name, items in pathology_variables_times['time2'].items():
    tv = TargetVariable(questionnaire_name, {'measurement': 'time2'}, items)
    df_time2 = tv.calculate_value(df_time2)



C:\Users\nogur\AppData\Local\Temp/ipykernel_17496/3639487288.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
df1 = df_time1[[questionnaire_name] + info_columns]
df2 = df_time2[[questionnaire_name] + info_columns]
df_ = pd.concat([df1, df2])
df_ = df_[df_.Group > 0]

In [10]:
df_.redcap_event_name.unique()

array(['intake_arm_1', 'control_5weeks_arm_1'], dtype=object)

In [11]:
w = df_[df_.redcap_event_name == 'control_5weeks_arm_1']

In [13]:
d = dtale.show(df_)
d.open_browser()

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked 

2023-06-20 13:06:30,918 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_args, *

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked 

2023-06-20 13:11:57,936 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\nogur\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\nogur\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_args, *

In [ ]:
time1 = ['intake_arm_1', 'pre_treatment_arm_1']

controls = [control_3month_arm_1, control_6month_arm_1]

time2 = ['5th_session_arm_1', 'control_5weeks_arm_1']


time3 = ['followup_3month_arm_1']


In [42]:
for i in list(df['redcap_event_name'].unique()):
    print(i)

followup_3month_arm_1
followup_12month_arm_1
control_9month_arm_1
control_12month_arm_1
5th_session_arm_1
post_treatment_arm_1
control_5weeks_arm_1
control_3month_arm_1
control_6month_arm_1
intake_arm_1
pre_treatment_arm_1
corona_arm_1
er_arm_1
er_one_week_arm_1


In [15]:
df.query("redcap_event_name == 'intake_arm_1'")[chameleon_cols]

chameleon_timestamp  chameleon_follow_stu  treatment_end_stu  \
114                  NaN                   NaN                NaN   
117                  NaN                   NaN                NaN   
119                  NaN                   NaN                NaN   
122                  NaN                   NaN                NaN   
131                  NaN                   NaN                NaN   
...                  ...                   ...                ...   
1944                 NaN                   NaN                NaN   
1945                 NaN                   NaN                NaN   
1946                 NaN                   NaN                NaN   
1947                 NaN                   NaN                NaN   
1948                 NaN                   NaN                NaN   

     treatment_end_2_stu visit_date_stu chameleon_ideation_stu  \
114                  NaN            NaN                    NaN   
117                  NaN            NaN                    NaN   
119                  NaN            NaN                    NaN   
122                  NaN            NaN                    NaN   
131                  NaN            NaN                    NaN   
...                  ...            ...                    ...   
1944                 NaN            NaN                    NaN   
1945                 NaN            NaN                    NaN   
1946                 NaN            NaN                    NaN   
1947                 NaN            NaN                    NaN   
1948                 NaN            NaN                    NaN   

      chameleon_behavior_stu  chameleon_attempt_stu  chameleon_nssi_stu  \
114                      NaN                    NaN                 NaN   
117                      NaN                    NaN                 NaN   
119                      NaN                    NaN                 NaN   
122                      NaN                    NaN                 NaN   
131                      NaN                    NaN                 NaN   
...                      ...                    ...                 ...   
1944                     NaN                    NaN                 NaN   
1945                     NaN                    NaN                 NaN   
1946                     NaN                    NaN                 NaN   
1947                     NaN                    NaN                 NaN   
1948                     NaN                    NaN                 NaN   

      chameleon_suicide_er_stu  ... chameleon_psychotherapy_2_stu___7  \
114                        NaN  ...                               0.0   
117                        NaN  ...                               0.0   
119                        NaN  ...                               0.0   
122                        NaN  ...                               0.0   
131                        NaN  ...                               0.0   
...                        ...  ...                               ...   
1944                       NaN  ...                               0.0   
1945                       NaN  ...                               0.0   
1946                       NaN  ...                               0.0   
1947                       NaN  ...                               0.0   
1948                       NaN  ...                               0.0   

     chameleon_psychotherapy_other_stu chameleon_medicine_stu  \
114                                NaN                    NaN   
117                                NaN                    NaN   
119                                NaN                    NaN   
122                                NaN                    NaN   
131                                NaN                    NaN   
...                                ...                    ...   
1944                               NaN                    NaN   
1945                               NaN                    NaN   
1946                

C:\Users\nogur\anaconda3\lib\site-packages\dtale\views.py:1918: FutureWarning:

The default value of regex will change from True to False in a future version.

